In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 
import pymssql
import seaborn as sns
import sklearn
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import train_test_split 
from xgboost import XGBRegressor
from sklearn import metrics

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_data():
    df = pd.DataFrame()
    try:
        con = pymssql.connect(user='username',password = 'password'
                 ,host='server',database='db',autocommit = True)
    except Exception as e:
        print('unable to make connection',e)
    cur=con.cursor()

    query = """
    -------------------------------------PAYMENTS AND CHARGES DATA----------------------------------
    ----------------------------charges data--------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_CHA') IS NOT NULL
    DROP TABLE #RCM_CHA
    Select pr.practice_code as Practice_Code,  MONTH(cc.DOE) as [Month], year(cc.DOE) as [Year],
    SUM(cc.Amount) as Charges_Sum
    into #RCM_CHA from Claim_Charges cc 
    join Claims c on
    cc.Claim_No = c.Claim_No and DATEDIFF(day, cc.DOE, GETDATE())<1200
    and isnull(c.Deleted,0)= 0 and isnull(cc.Deleted,0)= 0
    join Patient p on
    p.Patient_Account = c.Patient_Account
    and isnull(p.Deleted,0)= 0
    join PRACTICES pr on
    pr.Practice_Code = p.Practice_Code 
    AND ISNULL(P.DELETED,0)=0
    AND PR.IS_ACTIVE=1 AND ISNULL(PR.IS_TEST_PRACTICE,0)=0
    AND ISNULL(PR.EXCLUDE_FROM_BILLING_REPORTS,0) = 0
    AND PR.EMR_NAME NOT IN ('CHARTS PRO','PM STANDALONE','TESTINGPRACTICE')
    AND PR.PRACTICE_CODE NOT LIKE '9090%' AND PR.PRAC_NAME NOT LIKE '%TEST%'
    AND ISNULL(Pr.Deleted,0)=0
    group by pr.practice_code , MONTH(cc.DOE), YEAR(cc.DOE)
    -----------------------------------------------------------------------  
    ---------------------------------Payments data--------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_PAY') IS NOT NULL
    DROP TABLE #RCM_PAY
    Select pr.practice_code as Practice_Code,  MONTH(CP.Date_Entry) as [Month], year(CP.Date_Entry) as [Year],
    SUM(cp.Amount_Paid) as Paymnet_Sum
    into #RCM_PAY from Claim_Payments cp
    join Claims c on
    cP.Claim_No  = c.Claim_No and DATEDIFF(day, CP.Date_Entry, GETDATE())<1200
    and isnull(c.Deleted,0)= 0 and isnull(cp.Deleted,0)= 0
    join Patient p on
    p.Patient_Account = c.Patient_Account
    and isnull(p.Deleted,0)= 0
    join PRACTICES pr on
    pr.Practice_Code = p.Practice_Code 
    AND ISNULL(P.DELETED,0)=0
    AND PR.IS_ACTIVE=1 AND ISNULL(PR.IS_TEST_PRACTICE,0)=0
    AND ISNULL(PR.EXCLUDE_FROM_BILLING_REPORTS,0) = 0
    AND PR.EMR_NAME NOT IN ('CHARTS PRO','PM STANDALONE','TESTINGPRACTICE')
    AND PR.PRACTICE_CODE NOT LIKE '9090%' AND PR.PRAC_NAME NOT LIKE '%TEST%'
    AND ISNULL(Pr.Deleted,0)=0
    group by pr.practice_code , MONTH(cp.Date_Entry), YEAR(CP.Date_Entry)
    -----------------------------------------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_PCD') IS NOT NULL
    DROP TABLE #RCM_PCD
    Select C.practice_code, C.[Month],C.[Year],C.Charges_Sum,p.Paymnet_Sum
    into #RCM_PCD
    from #RCM_CHA C
    join #RCM_PAY P on
    c.Practice_Code = p.Practice_Code and c.[Month] = p.[Month] and c.[Year] = p.[Year]  
    -----------------------------------------------------------------------------------------------
    IF OBJECT_ID('TEMPDB..#RCM_DATA') IS NOT NULL
    DROP TABLE #RCM_DATA
    SELECT r.practice_code, r.[Month],r.[Year],r.Charges_Sum,r.Paymnet_Sum,
    a.AGING_0_30, a.AGING_31_60, a.AGING_61_90, a.AGING_91_120,a.AGING_120_PLUS  into #RCM_DATA
    from #RCM_PCD r
    join MONTH_END_AGING a on
    r.practice_code = a.PRACTICE_CODE and r.[Year] = a.CLOSE_YEAR and  r.[Month] = a.CLOSE_MONTH 
    order by practice_code, YEAR, MONTH 
	------------------------------------------------------------------------------------------------------------
   	IF OBJECT_ID('TEMPDB..#RCM_PREV') IS NOT NULL
    DROP TABLE #RCM_PREV
    SELECT practice_code, Month,Year,Charges_Sum,
    COALESCE(lag(Charges_Sum) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_Month_Charges,
    Paymnet_Sum, 
    COALESCE(lag(Paymnet_Sum) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_Month_Payments,
    AGING_0_30, AGING_31_60, AGING_61_90, AGING_91_120,AGING_120_PLUS  into #RCM_PREV
    from #RCM_DATA
    -----------------------------------------------------------------------------------------------------------------
   	IF OBJECT_ID('TEMPDB..#RCM_FIN') IS NOT NULL
    DROP TABLE #RCM_FIN
    SELECT practice_code, Month,Year,Charges_Sum,Prev_Month_Charges,
    COALESCE(lag(Prev_Month_Charges) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_2nd_Month_Charges,
    Paymnet_Sum, Prev_Month_Payments,
    COALESCE(lag(Prev_Month_Payments) over (partition by PRACTICE_CODE order by [YEAR],[MONTH]), 0) as Prev_2nd_Month_Payments,
    AGING_0_30, AGING_31_60, AGING_61_90, AGING_91_120,AGING_120_PLUS  into #RCM_FIN
    from #RCM_PREV
    
    Select * from #RCM_FIN order by practice_code, YEAR, MONTH 
    ---------------------------------------------------------------------------------------------------------------------------

    
     
    """ 
    
    try:
        cur.execute(query)
        df=pd.DataFrame(cur.fetchall(),columns=[x[0] for x in cur.description])
        df.columns=df.columns.str.upper()
        con.close()
    except:
        print('Error in reading data.....')
        con.close()
    return df

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# df = get_data()

In [6]:
# df.to_csv("YEARLY_PAY_CHAR.CSV", index = False)

In [7]:
data = pd.read_csv("YEARLY_PAY_CHAR.CSV")

In [8]:
data_copy = data.copy()

In [9]:
data.head()

,PRACTICE_CODE,MONTH,YEAR,CHARGES_SUM,PREV_MONTH_CHARGES,PREV_2ND_MONTH_CHARGES,PAYMNET_SUM,PREV_MONTH_PAYMENTS,PREV_2ND_MONTH_PAYMENTS,AGING_0_30,AGING_31_60,AGING_61_90,AGING_91_120,AGING_120_PLUS
0,100,10,2019,18120.72,0.00,0.00,7485.24,0.00,0.00,47723.15,5529.94,2041.04,2271.57,3988.1601
1,100,11,2019,98578.42,18120.72,0.00,39846.26,7485.24,0.00,36599.89,6967.96,1655.20,1061.77,4712.2001
2,100,12,2019,94319.79,98578.42,18120.72,37207.94,39846.26,7485.24,35010.79,6984.46,3668.20,3117.40,5870.2801
3,100,1,2020,77354.56,94319.79,98578.42,26831.85,37207.94,39846.26,42561.52,11326.57,2097.54,2608.87,4403.1902
4,100,2,2020,105143.63,77354.56,94319.79,29557.52,26831.85,37207.94,49423.63,15254.00,6792.06,1655.89,4735.5202


In [10]:
data['PC_RATIO'] = (data['PAYMNET_SUM']/data['CHARGES_SUM'])*100

In [11]:
data.fillna(0, inplace=True)
data.replace([np.inf, -np.inf], 0, inplace=True)

In [12]:
data = data.rename(columns = {'PRACTICE_CODE':'PRAC_CODE', 'CHARGES_SUM':'CHARG_SUM','PREV_MONTH_CHARGES':'PREV_M_CHARG','PREV_2ND_MONTH_CHARGES':'PREV_2ND_M_CHARG',
                      'PAYMNET_SUM':'PAY_SUM','PREV_MONTH_PAYMENTS':'PREV_M_PAY','PREV_2ND_MONTH_PAYMENTS':'PREV_2ND_M_PAY'})

In [13]:
data['PREV_3RD_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_2ND_M_CHARG'].shift(1)
data['PREV_4TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_3RD_M_CHARG'].shift(1)
data['PREV_5TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_4TH_M_CHARG'].shift(1)
data['PREV_6TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_5TH_M_CHARG'].shift(1)
data['PREV_7TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_6TH_M_CHARG'].shift(1)
data['PREV_8TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_7TH_M_CHARG'].shift(1)
data['PREV_9TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_8TH_M_CHARG'].shift(1)
data['PREV_10TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_9TH_M_CHARG'].shift(1)
data['PREV_11TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_10TH_M_CHARG'].shift(1)
data['PREV_12TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_11TH_M_CHARG'].shift(1)

data['PREV_13TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_12TH_M_CHARG'].shift(1)
data['PREV_14TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_13TH_M_CHARG'].shift(1)
data['PREV_15TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_14TH_M_CHARG'].shift(1)
data['PREV_16TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_15TH_M_CHARG'].shift(1)
data['PREV_17TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_16TH_M_CHARG'].shift(1)
data['PREV_18TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_17TH_M_CHARG'].shift(1)
data['PREV_19TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_18TH_M_CHARG'].shift(1)
data['PREV_20TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_19TH_M_CHARG'].shift(1)
data['PREV_21TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_20TH_M_CHARG'].shift(1)
data['PREV_22TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_21TH_M_CHARG'].shift(1)
data['PREV_23TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_22TH_M_CHARG'].shift(1)
data['PREV_24TH_M_CHARG'] = data.groupby('PRAC_CODE')['PREV_23TH_M_CHARG'].shift(1)

In [14]:
data['PREV_3RD_M_PAY'] = data.groupby('PRAC_CODE')['PREV_2ND_M_PAY'].shift(1)
data['PREV_4TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_3RD_M_PAY'].shift(1)
data['PREV_5TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_4TH_M_PAY'].shift(1)
data['PREV_6TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_5TH_M_PAY'].shift(1)
data['PREV_7TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_6TH_M_PAY'].shift(1)
data['PREV_8TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_7TH_M_PAY'].shift(1)
data['PREV_9TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_8TH_M_PAY'].shift(1)
data['PREV_10TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_9TH_M_PAY'].shift(1)
data['PREV_11TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_10TH_M_PAY'].shift(1)
data['PREV_12TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_11TH_M_PAY'].shift(1)

data['PREV_13TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_12TH_M_PAY'].shift(1)
data['PREV_14TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_13TH_M_PAY'].shift(1)
data['PREV_15TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_14TH_M_PAY'].shift(1)
data['PREV_16TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_15TH_M_PAY'].shift(1)
data['PREV_17TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_16TH_M_PAY'].shift(1)
data['PREV_18TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_17TH_M_PAY'].shift(1)
data['PREV_19TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_18TH_M_PAY'].shift(1)
data['PREV_20TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_19TH_M_PAY'].shift(1)
data['PREV_21TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_20TH_M_PAY'].shift(1)
data['PREV_22TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_21TH_M_PAY'].shift(1)
data['PREV_23TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_22TH_M_PAY'].shift(1)
data['PREV_24TH_M_PAY'] = data.groupby('PRAC_CODE')['PREV_23TH_M_PAY'].shift(1)

In [15]:
data['PREV_M_AGING_0_30'] = data.groupby('PRAC_CODE')['AGING_0_30'].shift(1)
data['PREV_2M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_M_AGING_0_30'].shift(1)
data['PREV_3M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_0_30'].shift(1)
data['PREV_4M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_0_30'].shift(1)
data['PREV_5M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_0_30'].shift(1)
data['PREV_6M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_0_30'].shift(1)
data['PREV_7M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_0_30'].shift(1)
data['PREV_8M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_0_30'].shift(1)
data['PREV_9M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_0_30'].shift(1)
data['PREV_10M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_0_30'].shift(1)
data['PREV_11M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_0_30'].shift(1)
data['PREV_12M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_0_30'].shift(1)

data['PREV_13M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_0_30'].shift(1)
data['PREV_14M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_0_30'].shift(1)
data['PREV_15M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_0_30'].shift(1)
data['PREV_16M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_0_30'].shift(1)
data['PREV_17M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_0_30'].shift(1)
data['PREV_18M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_0_30'].shift(1)
data['PREV_19M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_0_30'].shift(1)
data['PREV_20M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_0_30'].shift(1)
data['PREV_21M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_0_30'].shift(1)
data['PREV_22M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_0_30'].shift(1)
data['PREV_23M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_0_30'].shift(1)
data['PREV_24M_AGING_0_30'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_0_30'].shift(1)

In [16]:
data['PREV_M_AGING_31_60'] = data.groupby('PRAC_CODE')['AGING_31_60'].shift(1)
data['PREV_2M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_M_AGING_31_60'].shift(1)
data['PREV_3M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_31_60'].shift(1)
data['PREV_4M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_31_60'].shift(1)
data['PREV_5M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_31_60'].shift(1)
data['PREV_6M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_31_60'].shift(1)
data['PREV_7M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_31_60'].shift(1)
data['PREV_8M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_31_60'].shift(1)
data['PREV_9M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_31_60'].shift(1)
data['PREV_10M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_31_60'].shift(1)
data['PREV_11M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_31_60'].shift(1)
data['PREV_12M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_31_60'].shift(1)

data['PREV_13M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_31_60'].shift(1)
data['PREV_14M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_31_60'].shift(1)
data['PREV_15M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_31_60'].shift(1)
data['PREV_16M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_31_60'].shift(1)
data['PREV_17M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_31_60'].shift(1)
data['PREV_18M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_31_60'].shift(1)
data['PREV_19M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_31_60'].shift(1)
data['PREV_20M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_31_60'].shift(1)
data['PREV_21M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_31_60'].shift(1)
data['PREV_22M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_31_60'].shift(1)
data['PREV_23M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_31_60'].shift(1)
data['PREV_24M_AGING_31_60'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_31_60'].shift(1)

In [17]:
data['PREV_M_AGING_61_90'] = data.groupby('PRAC_CODE')['AGING_61_90'].shift(1)
data['PREV_2M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_M_AGING_61_90'].shift(1)
data['PREV_3M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_61_90'].shift(1)
data['PREV_4M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_61_90'].shift(1)
data['PREV_5M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_61_90'].shift(1)
data['PREV_6M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_61_90'].shift(1)
data['PREV_7M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_61_90'].shift(1)
data['PREV_8M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_61_90'].shift(1)
data['PREV_9M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_61_90'].shift(1)
data['PREV_10M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_61_90'].shift(1)
data['PREV_11M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_61_90'].shift(1)
data['PREV_12M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_61_90'].shift(1)

data['PREV_13M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_61_90'].shift(1)
data['PREV_14M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_61_90'].shift(1)
data['PREV_15M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_61_90'].shift(1)
data['PREV_16M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_61_90'].shift(1)
data['PREV_17M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_61_90'].shift(1)
data['PREV_18M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_61_90'].shift(1)
data['PREV_19M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_61_90'].shift(1)
data['PREV_20M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_61_90'].shift(1)
data['PREV_21M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_61_90'].shift(1)
data['PREV_22M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_61_90'].shift(1)
data['PREV_23M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_61_90'].shift(1)
data['PREV_24M_AGING_61_90'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_61_90'].shift(1)

In [18]:
data['PREV_M_AGING_91_120'] = data.groupby('PRAC_CODE')['AGING_91_120'].shift(1)
data['PREV_2M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_M_AGING_91_120'].shift(1)
data['PREV_3M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_91_120'].shift(1)
data['PREV_4M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_91_120'].shift(1)
data['PREV_5M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_91_120'].shift(1)
data['PREV_6M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_91_120'].shift(1)
data['PREV_7M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_91_120'].shift(1)
data['PREV_8M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_91_120'].shift(1)
data['PREV_9M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_91_120'].shift(1)
data['PREV_10M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_91_120'].shift(1)
data['PREV_11M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_91_120'].shift(1)
data['PREV_12M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_91_120'].shift(1)

data['PREV_13M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_91_120'].shift(1)
data['PREV_14M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_91_120'].shift(1)
data['PREV_15M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_91_120'].shift(1)
data['PREV_16M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_91_120'].shift(1)
data['PREV_17M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_91_120'].shift(1)
data['PREV_18M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_91_120'].shift(1)
data['PREV_19M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_91_120'].shift(1)
data['PREV_20M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_91_120'].shift(1)
data['PREV_21M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_91_120'].shift(1)
data['PREV_22M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_91_120'].shift(1)
data['PREV_23M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_91_120'].shift(1)
data['PREV_24M_AGING_91_120'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_91_120'].shift(1)

In [19]:
data['PREV_M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['AGING_120_PLUS'].shift(1)
data['PREV_2M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_M_AGING_120_PLUS'].shift(1)
data['PREV_3M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_2M_AGING_120_PLUS'].shift(1)
data['PREV_4M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_3M_AGING_120_PLUS'].shift(1)
data['PREV_5M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_4M_AGING_120_PLUS'].shift(1)
data['PREV_6M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_5M_AGING_120_PLUS'].shift(1)
data['PREV_7M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_6M_AGING_120_PLUS'].shift(1)
data['PREV_8M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_7M_AGING_120_PLUS'].shift(1)
data['PREV_9M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_8M_AGING_120_PLUS'].shift(1)
data['PREV_10M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_9M_AGING_120_PLUS'].shift(1)
data['PREV_11M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_10M_AGING_120_PLUS'].shift(1)
data['PREV_12M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_11M_AGING_120_PLUS'].shift(1)

data['PREV_13M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_12M_AGING_120_PLUS'].shift(1)
data['PREV_14M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_13M_AGING_120_PLUS'].shift(1)
data['PREV_15M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_14M_AGING_120_PLUS'].shift(1)
data['PREV_16M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_15M_AGING_120_PLUS'].shift(1)
data['PREV_17M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_16M_AGING_120_PLUS'].shift(1)
data['PREV_18M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_17M_AGING_120_PLUS'].shift(1)
data['PREV_19M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_18M_AGING_120_PLUS'].shift(1)
data['PREV_20M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_19M_AGING_120_PLUS'].shift(1)
data['PREV_21M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_20M_AGING_120_PLUS'].shift(1)
data['PREV_22M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_21M_AGING_120_PLUS'].shift(1)
data['PREV_23M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_22M_AGING_120_PLUS'].shift(1)
data['PREV_24M_AGING_120_PLUS'] = data.groupby('PRAC_CODE')['PREV_23M_AGING_120_PLUS'].shift(1)

In [20]:
data['PREV_M_PC_RATIO'] = data.groupby('PRAC_CODE')['PC_RATIO'].shift(1)
data['PREV_2M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_M_PC_RATIO'].shift(1)
data['PREV_3M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_2M_PC_RATIO'].shift(1)
data['PREV_4M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_3M_PC_RATIO'].shift(1)
data['PREV_5M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_4M_PC_RATIO'].shift(1)
data['PREV_6M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_5M_PC_RATIO'].shift(1)
data['PREV_7M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_6M_PC_RATIO'].shift(1)
data['PREV_8M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_7M_PC_RATIO'].shift(1)
data['PREV_9M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_8M_PC_RATIO'].shift(1)
data['PREV_10M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_9M_PC_RATIO'].shift(1)
data['PREV_11M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_10M_PC_RATIO'].shift(1)
data['PREV_12M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_11M_PC_RATIO'].shift(1)

data['PREV_13M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_12M_PC_RATIO'].shift(1)
data['PREV_14M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_13M_PC_RATIO'].shift(1)
data['PREV_15M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_14M_PC_RATIO'].shift(1)
data['PREV_16M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_15M_PC_RATIO'].shift(1)
data['PREV_17M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_16M_PC_RATIO'].shift(1)
data['PREV_18M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_17M_PC_RATIO'].shift(1)
data['PREV_19M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_18M_PC_RATIO'].shift(1)
data['PREV_20M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_19M_PC_RATIO'].shift(1)
data['PREV_21M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_20M_PC_RATIO'].shift(1)
data['PREV_22M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_21M_PC_RATIO'].shift(1)
data['PREV_23M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_22M_PC_RATIO'].shift(1)
data['PREV_24M_PC_RATIO'] = data.groupby('PRAC_CODE')['PREV_23M_PC_RATIO'].shift(1)

## Need to exclude month which is used as input data for prediction(e.g for Jan prediction we will give Dec as Input variable and will exclude it from training data)

In [161]:
data_train = data.loc[(data['YEAR']==2021) |(data['YEAR']==2023)|((data['YEAR']==2022) & (data['MONTH']!=12))]

In [162]:
# data_train = data.loc[((data['YEAR']==2022) & (data['MONTH']!=12))]

In [163]:
data_train = data_train[['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PAY_SUM','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
'AGING_0_30','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
'PREV_24M_AGING_0_30','AGING_31_60','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','AGING_61_90','PREV_M_AGING_61_90',
'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','AGING_91_120','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','AGING_120_PLUS','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
'PREV_24M_AGING_120_PLUS','PC_RATIO','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO']]

In [164]:
data_train.fillna(0, inplace=True)
data_train.replace([np.inf, -np.inf], 0, inplace=True)

In [165]:
data_train.head(100)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,AGING_0_30,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,AGING_31_60,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,AGING_61_90,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,AGING_91_120,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,AGING_120_PLUS,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PC_RATIO,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
15,100,1,2021,79764.30,61761.44,87013.06,128006.02,71746.25,62649.69,81547.41,51948.85,21518.43,29072.00,67294.84,105143.63,77354.56,9

## Charges Prediction

In [166]:
# Predictors=['PRAC_CODE','MONTH','YEAR','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
# 'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
# 'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
# 'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
# 'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30',
# 'PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
# 'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
# 'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
# 'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
# 'PREV_24M_AGING_0_30','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
# 'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
# 'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
# 'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
# 'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','PREV_M_AGING_61_90',
# 'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
# 'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
# 'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
# 'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
# 'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
# 'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
# 'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
# 'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
# 'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
# 'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
# 'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
# 'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
# 'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
# 'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
# 'PREV_24M_AGING_120_PLUS']
# TargetVariable=['CHARG_SUM']
# X=data_train[Predictors].values
# y=data_train[TargetVariable].values

In [167]:
Predictors=['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PAY_SUM','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
'AGING_0_30','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
'PREV_24M_AGING_0_30','AGING_31_60','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','AGING_61_90','PREV_M_AGING_61_90',
'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','AGING_91_120','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','AGING_120_PLUS','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
'PREV_24M_AGING_120_PLUS','PC_RATIO','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO']
TargetVariable=['CHARG_SUM']
X=data_train[Predictors].values
y=data_train[TargetVariable].values

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [169]:
xgb_regressor = xgb.XGBRegressor(n_estimators=1000,
    learning_rate=0.01,
    max_depth=12)

In [170]:
xgb_regressor.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=12, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [171]:
feature_importance = pd.DataFrame(xgb_regressor.feature_importances_)
feature_importance

,0
0,1.153680e-04
1,5.485363e-04
2,5.319739e-08
3,9.993361e-01
4,1.457350e-11
5,1.431024e-11
6,1.958375e-11
7,1.880461e-11
8,2.134068e-11
9,2.594092e-11


## PREDICTION PART

In [172]:
data.head(1)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,AGING_0_30,AGING_31_60,AGING_61_90,AGING_91_120,AGING_120_PLUS,PC_RATIO,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
0,100,10,2019,18120.72,0.0,0.0,7485.24,0.0,0.0,47723.15,5529.94,2041.04,2271.57,3988.1601,41.30763,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

## Leveling Number of Rows for Input and Test set

In [173]:
data_23 = data.loc[((data['YEAR']==2023) & (data['MONTH']==1))]['PRAC_CODE']
data_23 = list(data_23)
len(data_23)

419

In [174]:
#Input data to predict future values(Need to change every month as current month)
data_pred = data.loc[(data['YEAR']==2022) & (data['MONTH']==12) & (data['PRAC_CODE'].isin(data_23))]

In [175]:
data_pred.head(2)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,AGING_0_30,AGING_31_60,AGING_61_90,AGING_91_120,AGING_120_PLUS,PC_RATIO,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
35,100,12,2022,171942.21,169673.52,214233.14,72378.21,62685.85,62957.42,77345.72,28075.07,46316.49,34123.47,17831.4202,42.094498,23087

In [176]:
len(data_pred)

406

In [177]:
data_22 = data_pred.loc[((data_pred['YEAR']==2022) & (data_pred['MONTH']==12))]['PRAC_CODE']

In [178]:
data_22 = list(data_22)

In [179]:
len(data_22)

406

In [180]:
data_23_test = data.loc[((data['YEAR']==2023) & (data['MONTH']==1)) & (data['PRAC_CODE'].isin(data_22))]
len(data_23_test)

406

In [181]:
# Predict=['PRAC_CODE','MONTH','YEAR','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
# 'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
# 'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
# 'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
# 'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30',
# 'PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
# 'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
# 'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
# 'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
# 'PREV_24M_AGING_0_30','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
# 'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
# 'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
# 'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
# 'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','PREV_M_AGING_61_90',
# 'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
# 'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
# 'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
# 'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
# 'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
# 'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
# 'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
# 'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
# 'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
# 'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
# 'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
# 'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
# 'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
# 'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
# 'PREV_24M_AGING_120_PLUS']

In [182]:
data_pred.PRAC_CODE.dtypes

dtype('int64')

In [183]:
# data_pred = data_pred[['PRAC_CODE','MONTH','YEAR','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
# 'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
# 'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
# 'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
# 'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30',
# 'PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
# 'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
# 'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
# 'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
# 'PREV_24M_AGING_0_30','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
# 'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
# 'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
# 'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
# 'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','PREV_M_AGING_61_90',
# 'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
# 'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
# 'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
# 'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
# 'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
# 'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
# 'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
# 'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
# 'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
# 'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
# 'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
# 'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
# 'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
# 'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
# 'PREV_24M_AGING_120_PLUS']]

In [184]:
data_pred = data_pred[['PRAC_CODE','MONTH','YEAR','CHARG_SUM','PREV_M_CHARG','PREV_2ND_M_CHARG','PREV_3RD_M_CHARG','PREV_4TH_M_CHARG',
'PREV_5TH_M_CHARG','PREV_6TH_M_CHARG','PREV_7TH_M_CHARG','PREV_8TH_M_CHARG', 'PREV_9TH_M_CHARG', 'PREV_10TH_M_CHARG',
'PREV_11TH_M_CHARG','PREV_12TH_M_CHARG','PREV_13TH_M_CHARG','PREV_14TH_M_CHARG','PREV_15TH_M_CHARG','PREV_16TH_M_CHARG',
'PREV_17TH_M_CHARG','PREV_18TH_M_CHARG','PREV_19TH_M_CHARG','PREV_20TH_M_CHARG','PREV_21TH_M_CHARG','PREV_22TH_M_CHARG',
'PREV_23TH_M_CHARG','PREV_24TH_M_CHARG','PAY_SUM','PREV_M_PAY', 'PREV_2ND_M_PAY','PREV_3RD_M_PAY','PREV_4TH_M_PAY',
'PREV_5TH_M_PAY','PREV_6TH_M_PAY', 'PREV_7TH_M_PAY','PREV_8TH_M_PAY', 'PREV_9TH_M_PAY', 'PREV_10TH_M_PAY',
'PREV_11TH_M_PAY','PREV_12TH_M_PAY','PREV_13TH_M_PAY','PREV_14TH_M_PAY','PREV_15TH_M_PAY','PREV_16TH_M_PAY','PREV_17TH_M_PAY',
'PREV_18TH_M_PAY','PREV_19TH_M_PAY','PREV_20TH_M_PAY','PREV_21TH_M_PAY','PREV_22TH_M_PAY','PREV_23TH_M_PAY','PREV_24TH_M_PAY',
'AGING_0_30','PREV_M_AGING_0_30','PREV_2M_AGING_0_30', 'PREV_3M_AGING_0_30','PREV_4M_AGING_0_30','PREV_5M_AGING_0_30', 'PREV_6M_AGING_0_30',
'PREV_7M_AGING_0_30','PREV_8M_AGING_0_30', 'PREV_9M_AGING_0_30','PREV_10M_AGING_0_30','PREV_11M_AGING_0_30',
'PREV_12M_AGING_0_30','PREV_13M_AGING_0_30','PREV_14M_AGING_0_30','PREV_15M_AGING_0_30','PREV_16M_AGING_0_30','PREV_17M_AGING_0_30',
'PREV_18M_AGING_0_30','PREV_19M_AGING_0_30','PREV_20M_AGING_0_30','PREV_21M_AGING_0_30','PREV_22M_AGING_0_30','PREV_23M_AGING_0_30',
'PREV_24M_AGING_0_30','AGING_31_60','PREV_M_AGING_31_60','PREV_2M_AGING_31_60', 'PREV_3M_AGING_31_60', 'PREV_4M_AGING_31_60',
'PREV_5M_AGING_31_60','PREV_6M_AGING_31_60', 'PREV_7M_AGING_31_60','PREV_8M_AGING_31_60', 'PREV_9M_AGING_31_60',
'PREV_10M_AGING_31_60','PREV_11M_AGING_31_60', 'PREV_12M_AGING_31_60','PREV_13M_AGING_31_60','PREV_14M_AGING_31_60',
'PREV_15M_AGING_31_60','PREV_16M_AGING_31_60','PREV_17M_AGING_31_60','PREV_18M_AGING_31_60','PREV_19M_AGING_31_60','PREV_20M_AGING_31_60',
'PREV_21M_AGING_31_60','PREV_22M_AGING_31_60','PREV_23M_AGING_31_60','PREV_24M_AGING_31_60','AGING_61_90','PREV_M_AGING_61_90',
'PREV_2M_AGING_61_90', 'PREV_3M_AGING_61_90', 'PREV_4M_AGING_61_90', 'PREV_5M_AGING_61_90','PREV_6M_AGING_61_90',
'PREV_7M_AGING_61_90', 'PREV_8M_AGING_61_90', 'PREV_9M_AGING_61_90', 'PREV_10M_AGING_61_90','PREV_11M_AGING_61_90',
'PREV_12M_AGING_61_90','PREV_13M_AGING_61_90','PREV_14M_AGING_61_90','PREV_15M_AGING_61_90','PREV_16M_AGING_61_90',
'PREV_17M_AGING_61_90','PREV_18M_AGING_61_90','PREV_19M_AGING_61_90','PREV_20M_AGING_61_90','PREV_21M_AGING_61_90',
'PREV_22M_AGING_61_90','PREV_23M_AGING_61_90','PREV_24M_AGING_61_90','AGING_91_120','PREV_M_AGING_91_120','PREV_2M_AGING_91_120',
'PREV_3M_AGING_91_120', 'PREV_4M_AGING_91_120','PREV_5M_AGING_91_120','PREV_6M_AGING_91_120','PREV_7M_AGING_91_120',
'PREV_8M_AGING_91_120', 'PREV_9M_AGING_91_120', 'PREV_10M_AGING_91_120','PREV_11M_AGING_91_120','PREV_12M_AGING_91_120',
'PREV_13M_AGING_91_120','PREV_14M_AGING_91_120','PREV_15M_AGING_91_120','PREV_16M_AGING_91_120','PREV_17M_AGING_91_120',
'PREV_18M_AGING_91_120','PREV_19M_AGING_91_120','PREV_20M_AGING_91_120','PREV_21M_AGING_91_120','PREV_22M_AGING_91_120',
'PREV_23M_AGING_91_120','PREV_24M_AGING_91_120','AGING_120_PLUS','PREV_M_AGING_120_PLUS','PREV_2M_AGING_120_PLUS','PREV_3M_AGING_120_PLUS', 
'PREV_4M_AGING_120_PLUS','PREV_5M_AGING_120_PLUS','PREV_6M_AGING_120_PLUS','PREV_7M_AGING_120_PLUS', 'PREV_8M_AGING_120_PLUS',
'PREV_9M_AGING_120_PLUS','PREV_10M_AGING_120_PLUS','PREV_11M_AGING_120_PLUS', 'PREV_12M_AGING_120_PLUS','PREV_13M_AGING_120_PLUS',
'PREV_14M_AGING_120_PLUS','PREV_15M_AGING_120_PLUS','PREV_16M_AGING_120_PLUS','PREV_17M_AGING_120_PLUS','PREV_18M_AGING_120_PLUS',
'PREV_19M_AGING_120_PLUS','PREV_20M_AGING_120_PLUS','PREV_21M_AGING_120_PLUS','PREV_22M_AGING_120_PLUS','PREV_23M_AGING_120_PLUS',
'PREV_24M_AGING_120_PLUS','PC_RATIO','PREV_M_PC_RATIO','PREV_2M_PC_RATIO', 'PREV_3M_PC_RATIO', 'PREV_4M_PC_RATIO','PREV_5M_PC_RATIO', 
'PREV_6M_PC_RATIO', 'PREV_7M_PC_RATIO','PREV_8M_PC_RATIO', 'PREV_9M_PC_RATIO', 'PREV_10M_PC_RATIO','PREV_11M_PC_RATIO',
'PREV_12M_PC_RATIO','PREV_13M_PC_RATIO','PREV_14M_PC_RATIO','PREV_15M_PC_RATIO','PREV_16M_PC_RATIO','PREV_17M_PC_RATIO',
'PREV_18M_PC_RATIO','PREV_19M_PC_RATIO','PREV_20M_PC_RATIO','PREV_21M_PC_RATIO','PREV_22M_PC_RATIO',
'PREV_23M_PC_RATIO','PREV_24M_PC_RATIO']]

In [185]:
pre = xgb_regressor.predict(data_pred)

In [186]:
len(pre)

406

In [187]:
prediction_charg = pd.DataFrame(pre,columns=['PREDICTED_CHARGES'])

In [188]:
prediction_charg.head()

,PREDICTED_CHARGES
0,171894.203125
1,56110.554688
2,20617.128906
3,44026.386719
4,111029.937500


In [189]:
data = data.fillna(0)

In [190]:
actual = data_23_test

In [191]:
# actual = data.loc[((data['YEAR']==2022) & (data['MONTH']==12))].where(data['PRAC_CODE'].isin(data.loc[(data['YEAR']==2022) & (data['MONTH']==11)]))

In [192]:
actual['PRAC_CODE'].count()

406

In [193]:
actual.head()

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREV_M_CHARG,PREV_2ND_M_CHARG,PAY_SUM,PREV_M_PAY,PREV_2ND_M_PAY,AGING_0_30,AGING_31_60,AGING_61_90,AGING_91_120,AGING_120_PLUS,PC_RATIO,PREV_3RD_M_CHARG,PREV_4TH_M_CHARG,PREV_5TH_M_CHARG,PREV_6TH_M_CHARG,PREV_7TH_M_CHARG,PREV_8TH_M_CHARG,PREV_9TH_M_CHARG,PREV_10TH_M_CHARG,PREV_11TH_M_CHARG,PREV_12TH_M_CHARG,PREV_13TH_M_CHARG,PREV_14TH_M_CHARG,PREV_15TH_M_CHARG,PREV_16TH_M_CHARG,PREV_17TH_M_CHARG,PREV_18TH_M_CHARG,PREV_19TH_M_CHARG,PREV_20TH_M_CHARG,PREV_21TH_M_CHARG,PREV_22TH_M_CHARG,PREV_23TH_M_CHARG,PREV_24TH_M_CHARG,PREV_3RD_M_PAY,PREV_4TH_M_PAY,PREV_5TH_M_PAY,PREV_6TH_M_PAY,PREV_7TH_M_PAY,PREV_8TH_M_PAY,PREV_9TH_M_PAY,PREV_10TH_M_PAY,PREV_11TH_M_PAY,PREV_12TH_M_PAY,PREV_13TH_M_PAY,PREV_14TH_M_PAY,PREV_15TH_M_PAY,PREV_16TH_M_PAY,PREV_17TH_M_PAY,PREV_18TH_M_PAY,PREV_19TH_M_PAY,PREV_20TH_M_PAY,PREV_21TH_M_PAY,PREV_22TH_M_PAY,PREV_23TH_M_PAY,PREV_24TH_M_PAY,PREV_M_AGING_0_30,PREV_2M_AGING_0_30,PREV_3M_AGING_0_30,PREV_4M_AGING_0_30,PREV_5M_AGING_0_30,PREV_6M_AGING_0_30,PREV_7M_AGING_0_30,PREV_8M_AGING_0_30,PREV_9M_AGING_0_30,PREV_10M_AGING_0_30,PREV_11M_AGING_0_30,PREV_12M_AGING_0_30,PREV_13M_AGING_0_30,PREV_14M_AGING_0_30,PREV_15M_AGING_0_30,PREV_16M_AGING_0_30,PREV_17M_AGING_0_30,PREV_18M_AGING_0_30,PREV_19M_AGING_0_30,PREV_20M_AGING_0_30,PREV_21M_AGING_0_30,PREV_22M_AGING_0_30,PREV_23M_AGING_0_30,PREV_24M_AGING_0_30,PREV_M_AGING_31_60,PREV_2M_AGING_31_60,PREV_3M_AGING_31_60,PREV_4M_AGING_31_60,PREV_5M_AGING_31_60,PREV_6M_AGING_31_60,PREV_7M_AGING_31_60,PREV_8M_AGING_31_60,PREV_9M_AGING_31_60,PREV_10M_AGING_31_60,PREV_11M_AGING_31_60,PREV_12M_AGING_31_60,PREV_13M_AGING_31_60,PREV_14M_AGING_31_60,PREV_15M_AGING_31_60,PREV_16M_AGING_31_60,PREV_17M_AGING_31_60,PREV_18M_AGING_31_60,PREV_19M_AGING_31_60,PREV_20M_AGING_31_60,PREV_21M_AGING_31_60,PREV_22M_AGING_31_60,PREV_23M_AGING_31_60,PREV_24M_AGING_31_60,PREV_M_AGING_61_90,PREV_2M_AGING_61_90,PREV_3M_AGING_61_90,PREV_4M_AGING_61_90,PREV_5M_AGING_61_90,PREV_6M_AGING_61_90,PREV_7M_AGING_61_90,PREV_8M_AGING_61_90,PREV_9M_AGING_61_90,PREV_10M_AGING_61_90,PREV_11M_AGING_61_90,PREV_12M_AGING_61_90,PREV_13M_AGING_61_90,PREV_14M_AGING_61_90,PREV_15M_AGING_61_90,PREV_16M_AGING_61_90,PREV_17M_AGING_61_90,PREV_18M_AGING_61_90,PREV_19M_AGING_61_90,PREV_20M_AGING_61_90,PREV_21M_AGING_61_90,PREV_22M_AGING_61_90,PREV_23M_AGING_61_90,PREV_24M_AGING_61_90,PREV_M_AGING_91_120,PREV_2M_AGING_91_120,PREV_3M_AGING_91_120,PREV_4M_AGING_91_120,PREV_5M_AGING_91_120,PREV_6M_AGING_91_120,PREV_7M_AGING_91_120,PREV_8M_AGING_91_120,PREV_9M_AGING_91_120,PREV_10M_AGING_91_120,PREV_11M_AGING_91_120,PREV_12M_AGING_91_120,PREV_13M_AGING_91_120,PREV_14M_AGING_91_120,PREV_15M_AGING_91_120,PREV_16M_AGING_91_120,PREV_17M_AGING_91_120,PREV_18M_AGING_91_120,PREV_19M_AGING_91_120,PREV_20M_AGING_91_120,PREV_21M_AGING_91_120,PREV_22M_AGING_91_120,PREV_23M_AGING_91_120,PREV_24M_AGING_91_120,PREV_M_AGING_120_PLUS,PREV_2M_AGING_120_PLUS,PREV_3M_AGING_120_PLUS,PREV_4M_AGING_120_PLUS,PREV_5M_AGING_120_PLUS,PREV_6M_AGING_120_PLUS,PREV_7M_AGING_120_PLUS,PREV_8M_AGING_120_PLUS,PREV_9M_AGING_120_PLUS,PREV_10M_AGING_120_PLUS,PREV_11M_AGING_120_PLUS,PREV_12M_AGING_120_PLUS,PREV_13M_AGING_120_PLUS,PREV_14M_AGING_120_PLUS,PREV_15M_AGING_120_PLUS,PREV_16M_AGING_120_PLUS,PREV_17M_AGING_120_PLUS,PREV_18M_AGING_120_PLUS,PREV_19M_AGING_120_PLUS,PREV_20M_AGING_120_PLUS,PREV_21M_AGING_120_PLUS,PREV_22M_AGING_120_PLUS,PREV_23M_AGING_120_PLUS,PREV_24M_AGING_120_PLUS,PREV_M_PC_RATIO,PREV_2M_PC_RATIO,PREV_3M_PC_RATIO,PREV_4M_PC_RATIO,PREV_5M_PC_RATIO,PREV_6M_PC_RATIO,PREV_7M_PC_RATIO,PREV_8M_PC_RATIO,PREV_9M_PC_RATIO,PREV_10M_PC_RATIO,PREV_11M_PC_RATIO,PREV_12M_PC_RATIO,PREV_13M_PC_RATIO,PREV_14M_PC_RATIO,PREV_15M_PC_RATIO,PREV_16M_PC_RATIO,PREV_17M_PC_RATIO,PREV_18M_PC_RATIO,PREV_19M_PC_RATIO,PREV_20M_PC_RATIO,PREV_21M_PC_RATIO,PREV_22M_PC_RATIO,PREV_23M_PC_RATIO,PREV_24M_PC_RATIO
36,100,1,2023,154472.17,171942.21,169673.52,57483.82,72378.21,62685.85,73793.76,16510.93,13766.46,28924.67,33147.7602,37.213059,214233

In [194]:
actual_charg = actual[['PRAC_CODE','MONTH','YEAR','CHARG_SUM']]

In [195]:
actual_charg['PREDICTED_CHARGES'] = prediction_charg['PREDICTED_CHARGES'].to_list()

In [196]:
actual_charg['PREDICTED_CHARGES']= actual_charg['PREDICTED_CHARGES'].apply(lambda x:int(x))

In [197]:
our_pred = actual_charg.copy()

In [198]:
actual_charg['VAR_CHAR_%']=(((actual_charg['CHARG_SUM']-actual_charg['PREDICTED_CHARGES'])/actual_charg['PREDICTED_CHARGES'])*100).round(1)

In [199]:
# actual_charg.sort_values(by = 'VAR_CHAR_%')

In [200]:
finance = pd.read_excel("finance_prediction_jan.xlsx")

In [201]:
finance.head(1)

,Prac_Code,Charges,Payments
0,100,200000.0,78000.0


In [202]:
finance = finance.rename(columns = {'Prac_Code':'PRAC_CODE','Charges':'FINANCE_CHARGES','Payments':'FINANCE_PAYMENTS' })

In [203]:
finance = finance.fillna(0)

In [204]:
our_pred.head(2)

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREDICTED_CHARGES
36,100,1,2023,154472.17,171894
73,111,1,2023,47919.88,56110


In [205]:
all_pred_pay = our_pred.merge(finance, left_on='PRAC_CODE', right_on='PRAC_CODE', how = 'left')

In [206]:
all_pred_pay = all_pred_pay.fillna(0)

In [207]:
all_pred_pay.FINANCE_CHARGES = all_pred_pay.FINANCE_CHARGES.apply(int)

In [208]:
all_pred_pay = all_pred_pay.drop(columns = 'FINANCE_PAYMENTS')

In [209]:
all_pred_pay['VAR_CHAR_%']=(((all_pred_pay['CHARG_SUM']-all_pred_pay['PREDICTED_CHARGES'])/all_pred_pay['PREDICTED_CHARGES'])*100).round(1)

In [210]:
all_pred_pay['FIN_VAR_CHAR_%']=(((all_pred_pay['CHARG_SUM']-all_pred_pay['FINANCE_CHARGES'])/all_pred_pay['FINANCE_CHARGES'])*100).round(1)

In [211]:
all_pred_pay = all_pred_pay.loc[(all_pred_pay['PREDICTED_CHARGES']!=0)&(all_pred_pay['FINANCE_CHARGES']!=0)]

In [212]:
all_pred_pay.sort_values(by='VAR_CHAR_%')

,PRAC_CODE,MONTH,YEAR,CHARG_SUM,PREDICTED_CHARGES,FINANCE_CHARGES,VAR_CHAR_%,FIN_VAR_CHAR_%
85,1011154,1,2023,128.00,-56,20937,-328.6,-99.4
135,1011969,1,2023,-668.00,104373,99923,-100.6,-100.7
357,5110633,1,2023,0.00,867760,157171,-100.0,-100.0
314,2910006,1,2023,0.00,29867,18459,-100.0,-100.0
7,204,1,2023,480.02,120618,65372,-99.6,-99.3
105,1011468,1,2023,4251.00,66017,66400,-93.6,-93.6
337,5010100,1,2023,9700.00,80877,16540,-88.0,-41.4
268,2110074,1,2023,3885.61,29712,25666,-86.9,-84.9
239,1810002,1,2023,16950.05,121464,113461,-86.0,-85.1
145,1012158,1,2023,14250.00,91873,60000,-84.5,-76.2


In [213]:
# all_pred_pay.loc[all_pred_pay['PRAC_CODE'] == 1012714]

In [214]:
# all_pred_pay.to_csv("predicted_charges.csv")

In [215]:
len(all_pred_pay)

308

In [216]:
# data.loc[(data['MONTH']==12) & (data['YEAR']==2022)]

In [217]:
actuall = all_pred_pay['CHARG_SUM'].sum()
pred = all_pred_pay['PREDICTED_CHARGES'].sum()

In [218]:
(pred/actuall)*100

103.01107107552568

## Overall Practices Percentage

## Data Science Part

In [219]:
char80= all_pred_pay.loc[all_pred_pay['VAR_CHAR_%'].between(-20,20)]
char80['PREDICTED_CHARGES']= char80['PREDICTED_CHARGES'].apply(int)
char80 = char80.sort_values(by = 'VAR_CHAR_%', ascending=True).reset_index(drop=True)

In [220]:
char_70_80_Per= all_pred_pay.loc[(all_pred_pay['VAR_CHAR_%'].between(-30,-20.01))|(all_pred_pay['VAR_CHAR_%'].between(20.01,30))]
char_70_80_Per['PREDICTED_CHARGES']= char_70_80_Per['PREDICTED_CHARGES'].apply(int)
char_70_80_Per = char_70_80_Per.sort_values(by = 'VAR_CHAR_%', ascending=False).reset_index(drop=True)

In [221]:
char_50_70= all_pred_pay.loc[(all_pred_pay['VAR_CHAR_%'].between(-50,-29.999))|(all_pred_pay['VAR_CHAR_%'].between(30.01,50))]
char_50_70['PREDICTED_CHARGES']= char_50_70['PREDICTED_CHARGES'].apply(int)
char_50_70 = char_50_70.sort_values(by = 'VAR_CHAR_%', ascending=False).reset_index(drop=True)

In [222]:
char_b_50= all_pred_pay.loc[(all_pred_pay['VAR_CHAR_%']<-50) | (all_pred_pay['VAR_CHAR_%']>50)]
char_b_50['PREDICTED_CHARGES']= char_b_50['PREDICTED_CHARGES'].apply(int)
char_b_50 = char_b_50.sort_values(by = 'VAR_CHAR_%', ascending=False).reset_index(drop=True)

In [223]:
print("Total practices =",all_pred_pay['PRAC_CODE'].count())
print("Practices with 80% Plus Accuracy =",char80['PRAC_CODE'].count())
print("Practices with 70 to 80% Accuracy =",char_70_80_Per['PRAC_CODE'].count())
print("Practices with 50 to 70% Accuracy =",char_50_70['PRAC_CODE'].count())
print("Practices with below 50% Accuracy =",char_b_50['PRAC_CODE'].count())

Total practices = 308
Practices with 80% Plus Accuracy = 134
Practices with 70 to 80% Accuracy = 58
Practices with 50 to 70% Accuracy = 50
Practices with below 50% Accuracy = 66


In [224]:
141+39+63+65

308

## Finance Part 

In [225]:
char80_fin= all_pred_pay.loc[all_pred_pay['FIN_VAR_CHAR_%'].between(-20,20)]
char80_fin['FINANCE_CHARGES']= char80_fin['FINANCE_CHARGES'].apply(int)
char80_fin = char80_fin.sort_values(by = 'FIN_VAR_CHAR_%', ascending=True).reset_index(drop=True)

In [226]:
# char_b_50_fin

In [227]:
char_70_80_Per_fin= all_pred_pay.loc[(all_pred_pay['FIN_VAR_CHAR_%'].between(-30,-20.01))|(all_pred_pay['FIN_VAR_CHAR_%'].between(20.01,30))]
char_70_80_Per_fin['FINANCE_CHARGES']= char_70_80_Per_fin['FINANCE_CHARGES'].apply(int)
char_70_80_Per_fin = char_70_80_Per_fin.sort_values(by = 'FIN_VAR_CHAR_%', ascending=False).reset_index(drop=True)

In [228]:
char_50_70_fin= all_pred_pay.loc[(all_pred_pay['FIN_VAR_CHAR_%'].between(-50,-29.999))|(all_pred_pay['FIN_VAR_CHAR_%'].between(30.01,50))]
char_50_70_fin['FINANCE_CHARGES']= char_50_70_fin['FINANCE_CHARGES'].apply(int)
char_50_70_fin = char_50_70_fin.sort_values(by = 'FIN_VAR_CHAR_%', ascending=False).reset_index(drop=True)

In [229]:
char_b_50_fin= all_pred_pay.loc[(all_pred_pay['FIN_VAR_CHAR_%']<-50) | (all_pred_pay['FIN_VAR_CHAR_%']>50)]
char_b_50_fin['FINANCE_CHARGES']= char_b_50_fin['FINANCE_CHARGES'].apply(int)
char_b_50_fin = char_b_50_fin.sort_values(by = 'FIN_VAR_CHAR_%', ascending=False).reset_index(drop=True)

In [230]:
print("Total practices =",all_pred_pay['PRAC_CODE'].count())
print("Practices with 80% Plus Accuracy =",char80_fin['PRAC_CODE'].count())
print("Practices with 70 to 80% Accuracy =",char_70_80_Per_fin['PRAC_CODE'].count())
print("Practices with 50 to 70% Accuracy =",char_50_70_fin['PRAC_CODE'].count())
print("Practices with below 50% Accuracy =",char_b_50_fin['PRAC_CODE'].count())

Total practices = 308
Practices with 80% Plus Accuracy = 140
Practices with 70 to 80% Accuracy = 42
Practices with 50 to 70% Accuracy = 49
Practices with below 50% Accuracy = 77


In [231]:
140+42+49+77

308